In [1]:
#importing modules
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re


from langdetect import detect_langs
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer



In [2]:
# #twitter api keys
# auth = tweepy.OAuth1UserHandler(
#    )

# api = tweepy.API(auth, wait_on_rate_limit=True)


In [3]:
# search_words = "#wakandaforever OR #WakandaForever OR #blackpanther OR #WakandaForeverID OR #BlackPantherWakandaForever OR #BlackPanther OR #BlackPanther2"
# search_query = search_words + " -filter:retweets AND -filter:replies"


# limit = 100000
# tweets = tweepy.Cursor(api.search_tweets, q=search_query, tweet_mode='extended').items(limit)

# data=[]
# for tweet in tweets:
#     data.append([tweet.id,tweet.user.screen_name, tweet.created_at, tweet.full_text,tweet.user.location, tweet.retweet_count,tweet.favorite_count, tweet.user.followers_count, tweet.user.friends_count, tweet.user.verified, tweet.source])

In [4]:
# columns =['id','username','time_of_tweet', 'tweet', 'location', 'retweets', 'likes', 'followers', 'following', 'verified', 'tweet_source']
# bp_tweets=pd.DataFrame(data, columns=columns)
# bp_tweets.to_csv("BlackPanther-24-11.csv")

In [2]:
tweets = pd.read_csv('BlackPanther-24-11.csv')
tweets.sample(10)

,Unnamed: 0,id,username,time_of_tweet,tweet,location,retweets,likes,followers,following,verified,tweet_source
18273,18273,1593807036801769472,Blaqu3_B3rry,2022-11-19 03:23:03+00:00,Michael B. GOTDAMN JORDAN!!!!\n#WakandaForever!,FL ✈ ROK,0,0,1060,719,False,Twitter for iPhone
863,863,1595587629365575680,ALEXRAM48938815,2022-11-24 01:18:30+00:00,ORGULLO MEXICANO #NAMORA @MMabelCadena 🇲🇽🌊🌊 #W...,NaN,0,5,400,274,False,Twitter Web App
26205,26205,1593163562264166400,JamieAppsMedia,2022-11-17 08:46:07+00:00,With the introduction of Namor to the MCU in #...,Australia,1,0,397,450,False,Twitter for iPhone
28230,28230,1593029111273775104,Txm_San,2022-11-16 23:51:51+00:00,I want it back! \n#FreeTheBulge\n#ReleaseTheBu...,"Barcelona, España",0,0,355,664,False,Twitter for Android
26341,26341,1593149233020166145,IMilhaus,2022-11-17 07:49:11+00:00,Say it ain’t so…. 🤦🏾‍♂️ Wakanda Must Fall (Spo...,Free Man Of This City,0,1,235,180,False,Twitter for iPhone
5161,5161,1594943038442598406,JD0892,2022-11-22 06:37:07+00:00,I like #WakandaForever more during a second vi...,Florida,0,0,928,3047,False,Twitter for iPhone
10701,10701,1594399592719814656,MCU_Direct,2022-11-20 18:37:40+00:00,#BlackPanther legacy https://t.co/e5JGkeofNk,NaN,1408,13473,1260582,136,False,Buffer
27980,27980,1593042135598895105,EduGladiators,2022-11-17 00:43:37+00:00,🍿Join #EduGladiators SATURDAY &amp; chat movie...,NaN,2,4,8670,3914,False,Twitter for iPhone
18359,18359,1593801440186798081,astevey13,2022-11-19 03:00:49+00:00,Me waiting for Namor to take me to Talocan🫶🏻 #...,Space,1,4,950,440,False,Twitter for iPhone
31585,31585,1592871121032208384,Gourav96670923,2022-11-16 13:24:04+00:00,Please like share and subscribe to my channel ...,India,0,0,0,9,False,Twitter Web App


In [3]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35054 entries, 0 to 35053
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     35054 non-null  int64 
 1   id             35054 non-null  int64 
 2   username       35054 non-null  object
 3   time_of_tweet  35054 non-null  object
 4   tweet          35054 non-null  object
 5   location       24584 non-null  object
 6   retweets       35054 non-null  int64 
 7   likes          35054 non-null  int64 
 8   followers      35054 non-null  int64 
 9   following      35054 non-null  int64 
 10  verified       35054 non-null  bool  
 11  tweet_source   35054 non-null  object
dtypes: bool(1), int64(6), object(5)
memory usage: 3.0+ MB


In [4]:
tweets.describe()

,Unnamed: 0,id,retweets,likes,followers,following
count,35054.000000,3.505400e+04,35054.000000,35054.000000,3.505400e+04,35054.000000
mean,17526.500000,1.593928e+18,4.522194,37.315513,3.897041e+04,1241.136247
std,10119.362505,8.477788e+14,63.704964,630.220989,5.791979e+05,6060.701871
min,0.000000,1.592672e+18,0.000000,0.000000,0.000000e+00,0.000000
25%,8763.250000,1.593155e+18,0.000000,0.000000,1.080000e+02,149.000000
50%,17526.500000,1.593876e+18,0.000000,1.000000,4.150000e+02,420.000000
75%,26289.750000,1.594521e+18,0.000000,3.000000,1.547000e+03,1047.000000
max,35053.000000,1.595755e+18,4870.000000,51334.000000,4.083612e+07,301618.000000


In [5]:
#drop unnamed column
tweets.drop(columns='Unnamed: 0', inplace = True)

#fill in missing location with 'None'
tweets['location'].fillna('None', inplace=True)

#change the datatype of tweet_time to datetime 
tweets['time_of_tweet'] = tweets['time_of_tweet'].astype('datetime64')

In [6]:
def get_hash(tweet):
    '''This function collect all the hashtags in the tweet column'''
    tweet = re.findall(r'\#\w+', tweet)
    return " ".join(tweet)

In [7]:
tweets['hashtags'] = tweets['tweet'].apply(get_hash)


In [8]:
tweets.sample(10)

,id,username,time_of_tweet,tweet,location,retweets,likes,followers,following,verified,tweet_source,hashtags
2736,1595293253074665474,BLASTAMUS,2022-11-23 05:48:45,1. Most of them are white\n2. NONE OF THEM are...,"Miami, FL",0,2,449,614,False,Twitter for iPhone,#WakandaForever #BlackPanther #Marvel
24819,1593277499265294336,iamNagu26,2022-11-17 16:18:52,#WakandaForever was good!,"Ampang, Wilayah Persekutuan Ku",0,0,70,412,False,Twitter for Android,#WakandaForever
4408,1595084735763853313,OtisTrnr101,2022-11-22 16:00:10,🚨🚨Before I went to see #blackpantherwakandafor...,"Austin, TX",2,0,440,1209,False,Twitter for iPhone,#blackpantherwakandaforever #BlackPanther #Bla...
89,1595727961638174721,cassette_123,2022-11-24 10:36:07,マーベルレジェンド \nワカンダフォーエバー\nナキア リペイント \nbefore← →...,Sapporo,1,12,1953,1184,False,Twitter for Android,#marvellegends #repaint #BlackPanther #BlackPa...
19115,1593741243158519808,JRzthoughts,2022-11-18 23:01:37,Well #WakandaForever was ok for me… I felt a ...,The world is my playground,0,0,2628,1141,False,Twitter for iPhone,#WakandaForever
29315,1592980371809120256,vulture,2022-11-16 20:38:11,What #BlackPanther needed was a kiss. @roxana_...,"New York, NY",0,5,511951,993,True,SocialFlow,#BlackPanther
16634,1593978228078948353,browndages,2022-11-19 14:43:18,Help us get noticed by @TonyBakercomedy so we ...,"Columbus, OH",1,3,2319,48,False,Twitter for iPad,#healinsilence #blackrepresentation #blackpant...
17637,1593862481729839104,boundindia,2022-11-19 07:03:22,With the release of #BlackPanther Wakanda Fore...,"Mumbai, India",5,3,1037,983,False,Twitter for Android,#BlackPanther
14193,1594122736875487235,mrtwothousand,2022-11-20 00:17:32,The Heffa on tiktok that is bashing #BlackPant...,"Wichita, KS",0,0,2647,4033,False,Twitter for Android,#BlackPanther #wakandaforever #tiktok #Candace...
34548,1592696763185786880,oliver_frank665,2022-11-16 01:51:13,"É isso gente, um pouco desidratado pós pantera...",Itaúna - MG,0,2,278,198,False,Twitter for iPhone,#WakandaForever


In [10]:
hashtags = pd.Series(" ".join(tweets['hashtags']).split()).value_counts()
#hashtags.to_csv('hashtag_freq.csv')
hashtags[:10]

#WakandaForever                20990
#BlackPanther                   9316
#BlackPantherWakandaForever     5887
#wakandaforever                 1997
#Namor                          1746
#blackpanther                   1297
#Marvel                         1211
#BlackPanther2                   998
#WakandaForeverID                978
#MarvelStudios                   808
dtype: int64

In [15]:
def get_languages(tweet):
    try:
        tweet = detect_langs(tweet)
    except: 
        tweet = "nolang"
    tweet = str(tweet).split(':')[0][1:]
    return tweet

In [16]:
tweets['language'] = tweets['tweet'].apply(get_languages)

In [17]:
tweets.sample(10)

,id,username,time_of_tweet,tweet,location,retweets,likes,followers,following,verified,tweet_source,hashtags,language
12574,1594234626691981313,BigYodaWash,2022-11-20 07:42:09,Catch me Streaming Live On \n\nTwitch Wash718 ...,"New York, NY",1,0,48,42,False,Twitter for Android,#Gaming #Broadcast #Comedy #Anime #Party #Netw...,en
28256,1593027868464156672,Popcorn_For_One,2022-11-16 23:46:55,I’m home\nI’m txting everyone I love that I lo...,My local cinema,0,0,202,772,False,Twitter for iPhone,#WakandaForever #andor,en
22979,1593391246822744065,angievj22_,2022-11-17 23:50:51,"I'm obsessed with everything in this movie, th...",None,7,34,26,81,False,Twitter for Android,#WakandaForever #BlackPanther2 #BlackPantherWa...,en
19430,1593720119338967041,chiedzawith2ds,2022-11-18 21:37:40,Anyone who wanted T'Challa recast should stfu ...,"Johannesburg, South Africa",0,1,271,23,False,Twitter for Android,#Wakandaforever,en
3945,1595142681076039681,Rugdog,2022-11-22 19:50:26,Finally saw #WakandaForever and now I'm an emo...,NJ usa,0,6,2469,3003,False,Twitter for Android,#WakandaForever,en
14862,1594087747190300674,_raspberrypies,2022-11-19 21:58:30,ten film wzbudzil tyle emocji co zaden od dluz...,he/she,0,1,581,919,False,Twitter for iPhone,#WakandaForever,pl
30221,1592937071853047808,realmudzo,2022-11-16 17:46:07,#WakandaForever Theme Party @Moloko_Menlyn is ...,Pretoria / Johannesburg ZA,0,0,314,477,False,Twitter for Android,#WakandaForever #molokomenlyn #moloko4ever #mo...,en
18986,1593749906971295744,flanfatal,2022-11-18 23:36:02,Pensando en él (Namor en wakanda) #wakandaforever,tony stark defense squad,0,0,1708,299,False,Twitter for Android,#wakandaforever,sw
3774,1595165115602853888,tuckertoonie,2022-11-22 21:19:34,Safe and sound #BlackPantherWakandaForever htt...,Somewhere out there,0,8,311,162,False,Twitter for iPad,#BlackPantherWakandaForever,en
2485,1595344084784365571,cakestudio_ug,2022-11-23 09:10:44,#blackpanthercake \nIt's a #wakandaforever❤️ t...,Buganda Road flats block 661,0,0,747,1500,False,Twitter for Android,#blackpanthercake #wakandaforever #WakandaFore...,en


In [18]:
lang_code = pd.read_csv('language_codes.csv')
dict_lang_code = dict(zip(lang_code['alpha2'], lang_code['English']))
tweets['language'] = tweets['language'].replace(dict_lang_code)

In [19]:
tweets.language.value_counts()[:10]

English       24208
Spanish        3332
Danish         1483
Portuguese     1453
French          772
Indonesian      526
Japanese        474
Thai            471
Norwegian       371
Italian         324
Name: language, dtype: int64

In [20]:
def get_links(tweets):
    link = re.findall(r'((http|ftp|https):\/\/)(([\w.-]*)\.([\w]*))', tweets)
    if len(link) < 1:
        return 'No'
    else: return 'Yes'


In [21]:
tweets['contains_media'] = tweets['tweet'].apply(get_links)

In [22]:
tweets.contains_media.value_counts()

Yes    20872
No     14182
Name: contains_media, dtype: int64

In [23]:
cast = ['namor', 'shuri', 'ironheart', 'ramonda', 'namora', 
        'okoye', 'aneka', 'nakia', 'mbaku', 'attuma', 'everett', ]


def get_cast(tweets):
    tweets = tweets.lower()
    tokens = word_tokenize(tweets)
    cast_names = [name for name in tokens if name in cast]
    cast_names_title = [name.title() for name in cast_names]
    return " ".join(cast_names_title)

In [24]:
tweets['cast_name'] = tweets['tweet'].apply(get_cast)

In [25]:
names = pd.Series(" ".join(tweets['cast_name']).split()).value_counts()
# names.to_csv('names_freq.csv') ##save the generated dataframe
names[:10]

Namor        3953
Shuri        1985
Okoye         348
Namora        295
Ironheart     288
Nakia         220
Ramonda       216
Mbaku         202
Attuma        123
Aneka          53
dtype: int64

In [26]:
tweets['tweet'] = tweets['tweet'].replace('\n', ' ', regex=True)


In [27]:
sid = SentimentIntensityAnalyzer()
tweets['sentient_scores'] = tweets['tweet'].apply(sid.polarity_scores)
tweets['sentient_scores'] = tweets['sentient_scores'].apply(lambda x: x['compound'])

In [28]:
def get_sentiment(tweet):
    if tweet > 0:
        return "Positive"
    elif tweet == 0:
        return np.nan
    else: return "Negative"

In [29]:
tweets['sentiment'] = tweets['sentient_scores'].apply(get_sentiment)

In [30]:
tweets['sentiment'].value_counts()

Positive    12441
Negative     5168
Name: sentiment, dtype: int64

In [31]:
tweets['location'].value_counts()[:50]

None                         10470
Los Angeles, CA                495
United States                  384
London, England                179
México                         176
UK                             171
New York, NY                   168
Atlanta, GA                    164
India                          152
California, USA                144
Brasil                         139
Chicago, IL                    137
United Kingdom                 115
London                          99
España                          98
Canada                          98
Paris, France                   95
 Bonaire Islands 🏙🏖🏝 / DF       93
New York, USA                   88
Houston, TX                     86
São Paulo, Brasil               84
Brooklyn, NY                    81
Florida, USA                    80
USA                             78
France                          77
New York                        74
she/her                         72
Washington, DC                  71
Texas, USA          

In [32]:
def get_country(inc_c, case, cor_c):
    tweets.loc[tweets['c_location'].str.contains(inc_c, case=case), 'c_location'] = cor_c


In [41]:
tweets['c_location'] = tweets['location']


dicts = {'brazil':'Brazil', 'brasil':'Brazil', 'nigeria':'Nigeria', 'méxico': 'Mexico', 'france':'France', 'india':'India', 'london':'United Kingdom', 'UK':'United Kingdom', 'england':'United Kingdom','Bonaire':'Bonaire Islands', 'España':'Spain', 'thailand':'Thailand', 'Ohio':'United States', 'South Africa':'South Africa'}
for key, val in dicts.items():
    get_country(key, False, val)


list_us = ['USA', 'KS', 'FL', 'CA', 'GA', 'AZ', 'CO','IL', 'TX', 'PA', 'DC', 'TN', 'NY', 'WA', 'NYC', 'NC', 'NV', 'OH', 'LA', 'MA', 'Los Angeles' 'new york','New York', 'Los Angeles']
for states in list_us:
    get_country(states, True, 'United States')

tweets['c_location'].replace('None', np.nan, inplace=True)
tweets['c_location'].value_counts()[:10]

United States     5768
United Kingdom    1123
Brazil             666
Mexico             582
India              572
France             311
Spain              308
Nigeria            193
South Africa       190
Thailand           104
Name: c_location, dtype: int64

In [42]:
tweets['tweet_source'].value_counts()[:10]

Twitter for iPhone      15298
Twitter for Android     11002
Twitter Web App          5560
TweetDeck                 575
Twitter for iPad          331
Instagram                 289
Hootsuite Inc.            218
Buffer                    174
IFTTT                     141
Twitter Media Studio      139
Name: tweet_source, dtype: int64

In [44]:
tweets['verified'].value_counts()

False    33570
True      1484
Name: verified, dtype: int64

In [43]:
tweets.sample(20)

,id,username,time_of_tweet,tweet,location,retweets,likes,followers,following,verified,tweet_source,hashtags,language,contains_media,cast_name,sentient_scores,sentiment,c_location
10780,1594396582971072512,Kidzcoolit,2022-11-20 18:25:42,MONDAY ON Kidzcoolit: https://t.co/CbL6c7qluC ...,UK,3,3,18680,5022,False,Twitter for iPhone,#PokemonScarletandviolet #Transformers #BlackP...,English,Yes,,0.0000,NaN,United Kingdom
15510,1594049773145055237,SL_D3sign,2022-11-19 19:27:36,Seated for #BlackPantherWakandaForever,"Berlin, Germany",0,3,391,129,False,Twitter for Android,#BlackPantherWakandaForever,English,No,,0.0000,NaN,"Berlin, Germany"
24663,1593285676929351680,julivanespi,2022-11-17 16:51:21,I saw #WakandaForever last night. Great movie!,None,0,0,28,254,False,Twitter Web App,#WakandaForever,English,No,,0.6588,Positive,NaN
1095,1595554064141631489,Brown_Vador,2022-11-23 23:05:07,Merci à Ryan Coogler pour son minimum de trav...,None,0,0,344,340,False,Twitter for iPhone,#LiveAvecJessy #WakandaForever,French,No,,0.0000,NaN,NaN
11200,1594376192211263491,BadHeartMex,2022-11-20 17:04:40,Los gringos como no tienen un carajo de histor...,CDMX,0,0,628,1191,False,Twitter for Android,#WakandaForeverID,Spanish,No,,-0.2960,Negative,CDMX
10337,1594419754739142656,MarvelDialogues,2022-11-20 19:57:47,"""Today we don't fight for one life, we fight f...",Avengers Tower,0,0,3,0,False,Marvel Dialogue Bot,#blackPanther #marvel,English,No,,0.5216,Positive,Avengers Tower
27176,1593086583711727616,gustavochavesf,2022-11-17 03:40:14,Gente o que foi #WakandaForever ?,"Amarelo, Azul e Branco, Brasil",0,1,2486,1957,False,Twitter for Android,#WakandaForever,Portuguese,No,,0.0000,NaN,Brazil
30809,1592910779589611520,somespells,2022-11-16 16:01:39,finalmente stasera vado a vedere #WakandaForev...,221B Baker Street,0,18,3570,2391,False,Twitter for iPhone,#WakandaForever,Norwegian,Yes,,0.0000,NaN,221B Baker Street
32114,1592830074298191874,MikaRoss__,2022-11-16 10:40:57,"hoy es el día, esta tarde veré #WakandaForever 🤍",Madrid,0,5,991,1202,False,Twitter for Android,#WakandaForever,Spanish,No,,0.0000,NaN,Madrid
22710,1593411211172794368,MajorIssuesCBC,2022-11-18 01:10:11,This scene. Hear us talk about #BlackPantherW...,"New York, NY",0,0,208,382,False,Buffer,#BlackPantherWakandaForever #BlackPanther #Wak...,English,Yes,,0.5423,Positive,United States


In [35]:
tweets.to_csv('BlackPanther_Cleaned.csv')

In [31]:
cv = tweets['location'].replace(np.nan, "None")

In [32]:
cv[cv.str.contains('IL')][:60]


105               Crystal Lake, IL
409                    Chicago, IL
573                    Chicago, IL
720                    Chicago, IL
734          Elk Grove Village, IL
880     Chicago, IL and NW Suburbs
1142                   Chicago, IL
1256                   Chicago, IL
1526                  Chicago, IL.
1639                   Chicago, IL
1647                    BRAZILHOPE
1909          Obacht: SPOILERZONE!
2073                   Chicago, IL
2080                   Chicago, IL
2117                    Urbana, IL
2174                   Chicago, IL
2533          Obacht: SPOILERZONE!
2599                   Chicago, IL
2900                   Chicago, IL
3055                   Chicago, IL
3312                 CHICAGO,   IL
3793                   Chicago, IL
4021     Bottom of the map...FILA.
4073        CEBU CITY, PHILIPPINES
4188                   Chicago, IL
4335                   Chicago, IL
4475                   Chicago, IL
4608                   Chicago, IL
4766                